In [38]:
import pandas as pd
import random
import io
import numpy as np
import string
import MeCab 
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, RepeatVector
from keras.utils import np_utils
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from six.moves import cPickle
from keras import backend as K
from keras.utils import np_utils

In [18]:
BATCH_SIZE = 32 # Batch size for GPU
NUM_WORDS = 10000 # Vocab length
MAX_LEN = 20 # Padding length (# of words)
LSTM_EMBED = 8 # Number of LSTM nodes

In [8]:
mecab = MeCab.Tagger('-Owakati')

p = Path("/home/ifte/Downloads/mobicontrol_data/corpus_mobicontrol.csv")
df = pd.read_csv(p, header=0, usecols=["page", "text"])

In [9]:
# Keyword Matching
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

def greeting(sentence):
    """If user's input is a greeting, return a greeting response"""
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [10]:
import re

def clean_text(text):
    replaced_text = re.sub(r'[【】]', ' ', text)       # 【】の除去
    replaced_text = re.sub(r'[・_]', '', replaced_text)       # ・ の除去
    replaced_text = re.sub(r'[（）()]', ' ', replaced_text)     # （）の除去
    replaced_text = re.sub(r'[［］\[\]]', ' ', replaced_text)   # ［］の除去
    replaced_text = re.sub(r'[@＠]\w+', '', replaced_text)  # メンションの除去
    replaced_text = re.sub(r'https?:\/\/.*?[\r\n ]', '', replaced_text)  # URLの除去
    replaced_text = re.sub(r'　', ' ', replaced_text)  # 全角空白の除去
    replaced_text = re.sub(r'\d+', '', replaced_text) # 数字の除去
    replaced_text = re.sub(r'[-/。,、.=]', ' ', replaced_text)
    return replaced_text

In [11]:
stop_words_ja = []
STOPWORD_FILE = Path("/home/ifte/amiebot_project/amie-HelpBot/amie_helpbot/" + '/assets/learning/stopword_ja.txt')
with open(STOPWORD_FILE, encoding='utf-8') as fr:
    stop_words_ja = fr.read().splitlines()

In [12]:
y = df[["page"]]
X = df["text"].apply(lambda x: mecab.parse(x).strip("\n"))
df['parsed'] = X 

In [13]:
classes = df.page.unique()
c_size = 100

result = pd.DataFrame()

for i in list(classes):
    arr_index = df[df.page == i].parsed.values
    data = ''.join(list(arr_index))
    
    all_data = clean_text(data)
    
    arr_words = np.array(all_data.split())
    #print(arr_words) 
    
    num = arr_words.shape[0]//c_size
    full = c_size * num
    rest = arr_words.shape[0] - full
    
    pad = np.zeros([c_size-rest], dtype=int)
    sc = np.concatenate((arr_words,pad))
    features = sc.reshape(num+1, c_size)
    
    df_v = pd.DataFrame(features)
    df_v["cls"] = i
    
    result = pd.concat([result, df_v]) 

    
#print(result)
#result.to_csv('processed.csv')


# Splitting into fixed length however neural network works with fixed length input, 
# we need to define this length by getting rid or trimming inputs that are longer than this length
# But we dont want to trim any information hence the above approach will work well
#################################################
#MAX_LEN = 20
#train_data = train_data[train_data.Question.apply(lambda x: len(x.split())) < MAX_LEN]
#################################################
# data is cleaned which we did above
#train_data.Question = train_data.Question.apply(lambda x: (re.sub('[^\u0620-\uFEF0\s]', '', x)).strip())
#################################################
#We have to pad sequences that are shorter than MAX_LEN
#train_data = pad_sequences(train_data, padding='post', truncating='post', maxlen=MAX_LEN)

# Tokenization is the process of reading the text we have and creating a vocabulary based on some parameters, 
# then using this vocabulary we define an index where every word in the vocabulary has its ID.
#################################################
#tokenizer = Tokenizer(num_words=NUM_WORDS, lower=False)
#tokenizer.fit_on_texts(train_data["Question"].values)
#train_data = tokenizer.texts_to_sequences(train_data["Question"].values)

In [16]:
Data_cols = result.iloc[:,:-1]
tog = Data_cols.apply(lambda x: ' '.join(x), axis=1)
#tog = result.iloc[:,:].apply(lambda x: ' '.join(x), axis=1)
sent = list(tog.values)

In [14]:
result

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,cls
0,MobiControl,の,設定,順序,端末,グループ,の,作成,コンソール,と,...,の,環境,が,ある,場合,は,AD,FS,と,0
1,の,接続,プロファイル,を,作成,し,て,おく,全て,の,...,Google,アカウント,managed,Google,Play,アカウント,但し,Android,端末,0
2,を,Android,Plus,の,モード,で,設定,する,場合,は,...,名,の,型式,など,の,設定,オプション,が,ある,0
3,つ,の,端末,グループ,に対し,複数,の,端末,登録,ルール,...,の,プロファイル,へ,の,積,込,アプリ,の,端末,0
4,へ,の,配付,と,インストール,iOS,端末,を,除く,全て,...,対象,端末,OS,別,の,設定,方法,は,左側,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,これ,に,チェック,を,入れる,と,PC,が,MobiControl,サーバ,...,Disconnect,PC,から,MobiControl,サーバ,へ,の,接続,または,330
8,切断,の,アクション,を,執る,とき,に,この,ボタン,を,...,©,SOTI,Inc,Canada,Translated,by,Penetrate,of,Limits,330
9,Co,Ltd,Tokyo,タブ,説明,General,Device,Name,MobiControl,コンソール,...,が,ある,可能,性,が,ある,場合,ここ,に,330
10,チェック,を,入れ,ます,エージェント,の,挙動,を,ログ,に,...,さ,れ,ます,Servers,接続,可能,な,MobiControl,サーバ,330


In [20]:
tokenizer = Tokenizer(num_words=NUM_WORDS, lower=False)
tokenizer.fit_on_texts(tog.values)
train_data = tokenizer.texts_to_sequences(tog.values)

In [34]:
train_data = pad_sequences(train_data, padding='post', truncating='post', maxlen=101, value=0)

In [35]:
len(train_data)
train_data.shape

(6873, 101)

In [37]:
model = Sequential()
model.add(Embedding(NUM_WORDS, 200, input_length=101))
model.add(LSTM(LSTM_EMBED, dropout=0.2, recurrent_dropout=0.2, input_shape=(train_data.shape[1], NUM_WORDS)))
model.add(RepeatVector(train_data.shape[-1]))
model.add(LSTM(LSTM_EMBED, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model.add(Dense(NUM_WORDS, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
model.fit(train_data, np.expand_dims(train_data, -1), epochs=5, batch_size=BATCH_SIZE)

model.save("lstm-encoder.h5")

Epoch 1/5
6873/6873 [==============================] - 311s 45ms/step - loss: 7.4923
Epoch 2/5
6873/6873 [==============================] - 318s 46ms/step - loss: 4.3335
Epoch 3/5
6873/6873 [==============================] - 317s 46ms/step - loss: 2.7238
Epoch 4/5
6873/6873 [==============================] - 315s 46ms/step - loss: 2.3992
Epoch 5/5
6873/6873 [==============================] - 326s 47ms/step - loss: 2.3468


In [48]:
K.set_learning_phase(False)
#tokenizer = cPickle.load(open("lstm-autoencoder-tokenizer.pickle", "rb"))
# Read the encoder model
#model = load_model("lstm-encoder.h5")

In [49]:
# Create the encoding function
encode = K.function([model.input, K.learning_phase()], [model.layers[1].output])

In [50]:
Questions = np.squeeze(np.array(encode([train_data])))

In [57]:
question = input('Please enter a question: \n')
#question = stemmer.stem(question)
question = tokenizer.texts_to_sequences([question])
question = pad_sequences(question, padding='post', truncating='post', maxlen=101)
question = np.squeeze(encode([question]))

rank = cosine_similarity(question.reshape(1, -1), Questions)
top = np.argsort(rank, axis=-1).T[-5:]
   

Please enter a question: 
mobicontrol database


In [58]:
top


array([[2285],
       [2284],
       [2283],
       [2281],
       [6872]])

In [60]:
for item in range(len(top)):
    #print(data['Answer'].iloc[item].values[0])
    idx = top[item,0]
    
        # if the dataset file structure "Quesition | Answer" then use following line  
        #print(data['Answer'].iloc[item].values[0])
    print("PageID: ",result.iloc[idx,-1],"idx",idx,"=",tog.values[idx])
    print("\n ########## \n")

PageID:  120 idx 2285 = 端末 ユーザ の 本人 認証 無 接続 端末 対策 端末 証明 書 Chrome の Web フィルタ ランチャー ランチャー と テンプレート ランチャー と タグ クルマ 運転 中 の ランチャー ブラウザ の セキュリティ 対策 ブラウザ の プロ キシ 端末 の 機能 制限 Exchange for Gmail VPN 無線 LAN ブック マーク SOTI Hub SOTI Surf KNOX プロファイル KNOX ライセンス の 登録 シングルサインオン パス コード ブラックリスト ･ アプリ ファイア ウォール ブラウザ の 機能 制限 端末 の 機能 制限 E メール の 設定 MS Exchange ActiveSync IMAP POP NitroDesk Trouchdown VPN と アプリ の 紐 づけ 請求 書 公私 分割 簡易 モード プロファイル ウィルス 対策

 ########## 

PageID:  120 idx 2284 = Android Enterprise プロファイル 端末 グループ の 作成 コンソール と 端末 アイコン 設定 を する Android Plus プロファイル ウィルス 対策 端末 の ファイル 暗号 化 端末 ユーザ の 本人 認証 無 接続 端末 対策 端末 証明 書 アプリ の 起動 制限 ファィアウォール ランチャー の 登録 ランチャー と テンプレート ランチャー と タグ クルマ 運転 中 の ランチャー 発 着信 制限 端末 の 機能 制限 E メール の 設定 MS Exchange ActiveSync IMAP POP NitroDesk Trouchdown APN の 設定 VPN の 設定 無線 LAN ホット スポット 無線 LAN の 設定 ブック マーク SOTI Hub SOTI Surf 端末 マネージャー Android Enterprise プロファイル ウィルス 対策

 ########## 

PageID:  119 idx 2283 = が 利用 不可 に でき ない 画面 ロック を 外し た 状態 だ と 常に WiFi 局 を 探す 動作 を する ので バッテリ 消費 が 増える 